In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Задача 1
Написать функцию на языке Python, которая будет принимать в себя датасет и считать **retention** игроков (по дням от даты регистрации игрока).

### Загрузка исходных данных для первой задачи

In [2]:
reg_df  = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/BQ1-qD8o67JjHg', sep=';')
auth_df = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/c_50rJhKWZ_pCQ', sep=';')

### Описание данных в таблице reg_df

В таблице представлены данные о времени регистрации.

- **reg_ts** - timestamp с временем регистрации
- **uid** - столбец с **user id** пользователей

In [3]:
reg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


In [4]:
reg_df.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


### Описание данных в таблице auth_df

В таблице представлены данные о времени входа пользователей в игру.

- **auth_ts** - timestamp с временем захода пользователей в игру
- **uid** - столбец с **user id** пользователей

In [5]:
auth_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [6]:
auth_df.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


### Привидение типов
Так как в данных даты представлены в формате **timestamp**, то рационально перевести тип данных в **datetime** и оставить только даты, исключив время совершаемого действия, чтобы было проще воспринимать информацию и строить когортный анализ.

In [7]:
reg_df['reg_ts'] = pd.to_datetime(reg_df.reg_ts, unit='s')
reg_df['reg_ts'] = reg_df.reg_ts.apply(lambda x: x.date())

In [8]:
auth_df['auth_ts'] = pd.to_datetime(auth_df.auth_ts, unit='s')
auth_df['auth_ts'] = auth_df.auth_ts.apply(lambda x: x.date())

### Объединение датафреймов
Для выполнения исходной задачи необходимо объединить имеющиеся датафреймы. Сделаем это посредством **left join** таблицы с датами регистрации к таблице с данными о входе в приложение по колонке **uid**.

In [9]:
retention_data = auth_df.merge(reg_df, how='left', on='uid')

### Требуемая функция

In [10]:
def cohort_analysis(data, start_cohort, end_cohort, start_period, end_period):
    
    """
       Функция реализует метод построения когорт для изучения retention пользователей приложения по дням.
       
       Параметры
       ---------
       data: pandas DataFrame
           Датасет
       
       start_cohort: string
           Строка, содержащая когорту (дату формата гггг-мм-дд), которая будет в первой строке таблицы
       
       end_cohort:   string
           Строка, содержащая когорту (дату формата гггг-мм-дд), которая будет в последней строке таблицы
           
       start_period: int
           Число, содержащее в себе начальное для анализа количество дней, прошедших со дня регистрации
       
       end_period:   int
           Число, содержащее в себе корнечное для анализа количество дней, прошедших со дня регистрации    
    """
    
    # Привидем типы данных для последующей фильтрации по датам
    start_cohort = pd.to_datetime(start_cohort)
    end_cohort = pd.to_datetime(end_cohort)
    
    # Группируем данные по когортам и датам авторизации
    # Считаем количество пользователей, совершивших авторизацию, в каждом периоде по каждой когорте
    cohort_df = retention_data \
        .groupby(['reg_ts', 'auth_ts'], as_index=False) \
        .agg(n_users=('uid', 'nunique')) \
        .rename(columns={'reg_ts': 'cohort'})
    
    # Создадим колонку, в которой найдем разницу в днях между датой регистрации и датой авторизации
    cohort_df['period_num'] = (cohort_df.auth_ts - cohort_df.cohort).dt.days
    
    # Преобразуем данные в сводную таблицу
    cohort_pivot = cohort_df.pivot_table(index   = 'cohort',        # Строки - когорты
                                         columns = 'period_num',    # Колонки - периоды (разница между когортой)
                                         values  = 'n_users')       # Значения - количество пользователей, вошедших в игру
    
    # Создаем pdSeries с количеством зарегистрировавшихся пользователей, в когорте
    cohort_size = cohort_pivot.iloc[:,0]
    
    # Рассчитываем retention: долю атворизовавшихся пользователей в периоде относительно количества пользователей в когорте
    retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)
    
    # Фильтруем данные по интересующим нас когортам и периодам
    retention_matrix = retention_matrix.loc[start_cohort:end_cohort, start_period:end_period]
    
    # Наводим марафет
    cohort_style = (retention_matrix
            .style
            .set_caption('User retention by cohort')  # Добавляем подпись
            .background_gradient(cmap='summer')       # Раскрашиваем ячейки по столбцам
            .highlight_null('white')                  # Делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))             # Числа форматируем как проценты, NaN заменяем на пустоту
    
    return cohort_style

### Результат работы функции
Построим когортный анализ для когорт с **2020-09-10** по **2020-09-23** и в периоде с **дня регистрации** по **13** день пользования приложением.

In [11]:
cohort_analysis(retention_data, '2020-09-10', '2020-09-23', 0, 13)

period_num,0,1,2,3,4,5,6,7,8,9,10,11,12,13
cohort,,,,,,,,,,,,,,
2020-09-10,100.00%,2.05%,4.29%,4.79%,4.23%,6.77%,6.28%,7.09%,4.85%,4.54%,5.66%,4.72%,5.22%,3.60%
2020-09-11,100.00%,1.30%,4.03%,4.53%,5.65%,5.71%,6.70%,6.33%,4.40%,4.65%,5.40%,5.33%,3.29%,
2020-09-12,100.00%,1.92%,3.84%,4.71%,4.65%,6.07%,6.57%,5.76%,5.27%,5.14%,5.70%,2.54%,,
2020-09-13,100.00%,1.98%,4.08%,4.26%,5.13%,6.55%,6.49%,4.94%,4.70%,4.94%,3.58%,,,
2020-09-14,100.00%,2.22%,3.83%,4.51%,5.31%,6.49%,6.98%,6.30%,4.45%,2.90%,,,,
2020-09-15,100.00%,2.22%,4.07%,4.56%,6.04%,5.61%,6.90%,5.67%,2.46%,,,,,
2020-09-16,100.00%,1.91%,4.25%,4.62%,4.68%,6.77%,6.95%,3.38%,,,,,,
2020-09-17,100.00%,1.66%,4.30%,4.55%,5.47%,6.27%,4.24%,,,,,,,
2020-09-18,100.00%,1.90%,4.54%,4.23%,5.21%,3.74%,,,,,,,,
